In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from langdetect import detect
import seaborn as sns

In [56]:
# Import files

In [57]:
coursera_courses = pd.read_csv("./Datasets/Moocs/Coursera_courses.csv")
print(coursera_courses.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 623 entries, 0 to 622
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         623 non-null    object
 1   institution  623 non-null    object
 2   course_url   623 non-null    object
 3   course_id    623 non-null    object
dtypes: object(4)
memory usage: 19.6+ KB
None


In [58]:
coursera_reviews = pd.read_csv("./Datasets/Moocs/Coursera_reviews.csv")
print(coursera_reviews.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454711 entries, 0 to 1454710
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   reviews       1454571 non-null  object
 1   reviewers     1454711 non-null  object
 2   date_reviews  1454711 non-null  object
 3   rating        1454711 non-null  int64 
 4   course_id     1454711 non-null  object
dtypes: int64(1), object(4)
memory usage: 55.5+ MB
None


# Check for duplicates

In [59]:
# Check for duplicates in coursera_courses dataframe
print("Duplicates in Coursera_courses:", coursera_courses.duplicated().sum())



# Check for duplicates in coursera_reviews dataframe
print("Duplicates in Coursera_reviews:", coursera_reviews.duplicated().sum())

Duplicates in Coursera_courses: 0
Duplicates in Coursera_reviews: 934764


# Drop duplicates in reviews

In [60]:
coursera_reviews.drop_duplicates(inplace=True)

In [61]:
print(coursera_reviews.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519947 entries, 0 to 1454644
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   reviews       519891 non-null  object
 1   reviewers     519947 non-null  object
 2   date_reviews  519947 non-null  object
 3   rating        519947 non-null  int64 
 4   course_id     519947 non-null  object
dtypes: int64(1), object(4)
memory usage: 23.8+ MB
None


In [62]:
coursera_reviews['num_reviews'] = coursera_reviews.groupby('course_id')['course_id'].transform('count')

# Drop duplicates in "coursera_reviews" based on "course_id", "reviews", and "reviewers" columns
coursera_reviews = coursera_reviews.drop_duplicates(subset=['course_id', 'reviews', 'reviewers'])

# Print for checking info
print(coursera_reviews.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 519928 entries, 0 to 1454644
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   reviews       519872 non-null  object
 1   reviewers     519928 non-null  object
 2   date_reviews  519928 non-null  object
 3   rating        519928 non-null  int64 
 4   course_id     519928 non-null  object
 5   num_reviews   519928 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 27.8+ MB
None


### Now drop duplicates by "curse_id" keeping the unique values that are representative of the avrg. rating takinng the most recent value for date

In [63]:
# Group the dataframe by course_id and calculate the mean rating for each group
mean_ratings = coursera_reviews.groupby('course_id')['rating'].mean()

# Create a new column 'mean_rating' in cursera_reviews that contains the mean rating for each course
coursera_reviews['mean_rating'] = coursera_reviews['course_id'].map(mean_ratings)

# Create a new column 'rating_diff' that contains the difference between the rating and the mean rating for each row
coursera_reviews['rating_diff'] = abs(coursera_reviews['rating'] - coursera_reviews['mean_rating'])

#### Sort the dataframe by course_id and date_reviews
coursera_reviews = coursera_reviews.sort_values(by=['course_id', 'date_reviews'], ascending=[True, False])

#### Drop duplicates of unique values in the column "course_id"
coursera_reviews = coursera_reviews.drop_duplicates(subset='course_id', keep='first')

In [64]:
# Print for checking info
print(coursera_reviews.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 604 entries, 1090514 to 1095185
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   reviews       604 non-null    object 
 1   reviewers     604 non-null    object 
 2   date_reviews  604 non-null    object 
 3   rating        604 non-null    int64  
 4   course_id     604 non-null    object 
 5   num_reviews   604 non-null    int64  
 6   mean_rating   604 non-null    float64
 7   rating_diff   604 non-null    float64
dtypes: float64(2), int64(2), object(4)
memory usage: 42.5+ KB
None


# Detect and generate language

In [65]:
from langdetect import detect

def detect_language(text):
    try:
        return detect(text)
    except:
        return "Error"
    
coursera_reviews['language'] = coursera_reviews[['reviews', 'course_id']].applymap(detect_language).mode(axis=1)[0]


# Mershing the two dataframes into one called "coursera_clean"

In [66]:
coursera_clean= pd.merge(coursera_reviews, coursera_courses, on='course_id')

In [67]:
print(coursera_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 604 entries, 0 to 603
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   reviews       604 non-null    object 
 1   reviewers     604 non-null    object 
 2   date_reviews  604 non-null    object 
 3   rating        604 non-null    int64  
 4   course_id     604 non-null    object 
 5   num_reviews   604 non-null    int64  
 6   mean_rating   604 non-null    float64
 7   rating_diff   604 non-null    float64
 8   language      604 non-null    object 
 9   name          604 non-null    object 
 10  institution   604 non-null    object 
 11  course_url    604 non-null    object 
dtypes: float64(2), int64(2), object(8)
memory usage: 61.3+ KB
None


In [68]:
print(coursera_clean)

                                               reviews         reviewers  \
0    The course provided a new perspective on the n...      By Richard J   
1    기업에서 관리되는 접근제어 방식 중 다양한 모델과 장치들을 알게 되었습니다. 초심자...       By PUREUM W   
2         Excellent thoughts and concepts presented.    By Shivanand R K   
3    This is just a basic Python programming course...   By Truong T T H   
4    Fue un curso muy padre y entendible para cualq...       By Brenda I   
..                                                 ...               ...   
599  Learnt so much on this subject! Amazing course...       By Benita A   
600                                   great experience    By GAYATRI S T   
601  It is a course which teaches you punctuality, ...         By Ayan C   
602  Leaves you better than you began. Thank you, P...      By SHANIYA B   
603  An excellent course that provides the most imp...        By Bryan C   

     date_reviews  rating                                          course_id  \
0    Se

# Drop unecesary columns and change column "name" for "title" and "course_id" for "subject"

In [69]:
coursera_clean = coursera_clean.drop(["mean_rating", "rating_diff"], axis=1).rename(columns={"name": "title", "course_id": "subject"})

In [70]:
print(coursera_clean.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 604 entries, 0 to 603
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   reviews       604 non-null    object
 1   reviewers     604 non-null    object
 2   date_reviews  604 non-null    object
 3   rating        604 non-null    int64 
 4   subject       604 non-null    object
 5   num_reviews   604 non-null    int64 
 6   language      604 non-null    object
 7   title         604 non-null    object
 8   institution   604 non-null    object
 9   course_url    604 non-null    object
dtypes: int64(2), object(8)
memory usage: 51.9+ KB
None


# add the column "price" and fill it with coursera subscription for proffesional certificates "$39 USD per month"
# also add coolumn platform and level

In [71]:
coursera_clean["price"] = "39"
coursera_clean["level"] = "unknown"
coursera_clean["platform"] = "coursera"

# Transform "price" type into "int"

In [72]:
coursera_clean['price'] = coursera_clean['price'].astype(int)


# Drop unecesary columns and order them

In [73]:
columns_to_keep = ['price', 'language', 'level', 'rating', 'title', 'institution', 'subject', 'platform']
coursera_clean = coursera_clean[columns_to_keep]

# Export cleaned files

In [74]:
# Export as .csv
coursera_clean.to_csv('./Datasets/Cleaned/coursera_clean.csv', index=False)

# Export as .xlsx
coursera_clean.to_excel('./Datasets/Cleaned/coursera_clean.xlsx', index=False)
